# Obtaining Our Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

Your boss gives you a general description of some of the datasets at your disposal for analyzing weekly store sales. They're eventually looking for you to build a model to help determine what factors impact sales, and model future sales forecasting for business planning.  
  
Most of the properietary store data sits in the company sql database, accessible by all managers and above. The database is called **Walmart.db** Your boss provides you with the following basic schema:  

<img src='db_schema.jpg' width=500>  

She then tells you that she's put together a second dataset on general economy statistics for the various dates that she would also like you to incorporate in your analysis. That data, she says, is stored in a file **economy_data.csv**.

As a first step in creating your model for providing recommendations and projections, load and synthesize these disperate datasets into a singular unified DataFrame. Then save your results to a file **Merged_Store_Data.csv**.

Make sure you check the various data types and merge appropriately.

## Import necessary libraries and instantiate sql cursor

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

connection = sql.connect('Walmart.db')
curs = connection.cursor()

## Import economy_data and rename as necessary

In [2]:
econ_df = pd.read_csv('economy_data.csv')
econ_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [3]:
econ_df.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000


In [4]:
econ_df.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

It looks like there is some mising data, but I'll wait to clean until it's necessary. For now, I think that it is most likely that the data will be merged on the date. The Sales and Store Details data will be merged on store number. Let's import those and take a look.

## Import Sales and store_details data as one dataframe merged on store number

In [5]:
curs.execute('''
            select * 
            from Sales
            left join
                store_details
            using(Store)
            ''')
sales_df = pd.DataFrame(curs.fetchall())
sales_df.columns = [i[0] for i in curs.description]
sales_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315
3,1,1,2010-02-26,19403.54,False,A,151315
4,1,1,2010-03-05,21827.90,False,A,151315


## Merge Sales and Economy Dataframe on Date, Store index

In [6]:
sales_df['Date'] = pd.to_datetime(sales_df.Date)
sales_df.set_index(['Date', 'Store'], inplace= True)
sales_df.head()

,,Dept,Weekly_Sales,IsHoliday,Type,Size
Date,Store,,,,,
2010-02-05,1,1,24924.50,False,A,151315
2010-02-12,1,1,46039.49,True,A,151315
2010-02-19,1,1,41595.55,False,A,151315
2010-02-26,1,1,19403.54,False,A,151315
2010-03-05,1,1,21827.90,False,A,151315


In [7]:
econ_df['Date'] = pd.to_datetime(econ_df.Date)
econ_df.set_index(['Date', 'Store'], inplace= True)
econ_df.head()

,,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
Date,Store,,,,,,,,,,
2010-02-05,1,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
2010-02-12,1,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2010-02-19,1,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
2010-02-26,1,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
2010-03-05,1,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [8]:
merged_df = pd.merge(sales_df, econ_df, how = 'inner', left_index = True, right_index = True, suffixes = ['_Sales', '_Econ'] )
merged_df.head()

Dept  Weekly_Sales IsHoliday_Sales Type    Size  \
Date       Store                                                    
2010-02-05 1         1      24924.50           False    A  151315   
           1         2      50605.27           False    A  151315   
           1         3      13740.12           False    A  151315   
           1         4      39954.04           False    A  151315   
           1         5      32229.38           False    A  151315   

                  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  \
Date       Store                                                             
2010-02-05 1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   

                  MarkDown4  MarkDown5         CPI  Unemployment  \
Date       Store                                                   
2010-02-05 1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   

                  IsHoliday_Econ  
Date       Store                  
2010-02-05 1               False  
           1               False  
           1               False  
           1               False  
           1               False

In [9]:
# There are two columns for IsHoliday. If these are equivalent, then I'll simply drop one
merged_df.replace(to_replace = ['False', 'True'], value = [0,1])
merged_df.replace({'IsHoliday_Sales': {'True': True, 'False': False}}, inplace = True)
merged_df.head()

Dept  Weekly_Sales  IsHoliday_Sales Type    Size  \
Date       Store                                                     
2010-02-05 1         1      24924.50            False    A  151315   
           1         2      50605.27            False    A  151315   
           1         3      13740.12            False    A  151315   
           1         4      39954.04            False    A  151315   
           1         5      32229.38            False    A  151315   

                  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  \
Date       Store                                                             
2010-02-05 1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   
           1            42.31       2.572        NaN        NaN        NaN   

                  MarkDown4  MarkDown5         CPI  Unemployment  \
Date       Store                                                   
2010-02-05 1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   
           1            NaN        NaN  211.096358         8.106   

                  IsHoliday_Econ  
Date       Store                  
2010-02-05 1               False  
           1               False  
           1               False  
           1               False  
           1               False

In [12]:
sum(merged_df.IsHoliday_Econ == merged_df.IsHoliday_Sales) == merged_df.IsHoliday_Econ.count()

True

Given that the holidays match up, I'll drop one of the holiday columns and rename the other one appropriately

In [13]:
merged_df.drop(labels = 'IsHoliday_Econ', axis = 1, inplace = True)
merged_df.rename(columns={'IsHoliday_Sales':'IsHoliday'}, inplace=True)
merged_df.head()

Dept  Weekly_Sales  IsHoliday Type    Size  Temperature  \
Date       Store                                                            
2010-02-05 1         1      24924.50      False    A  151315        42.31   
           1         2      50605.27      False    A  151315        42.31   
           1         3      13740.12      False    A  151315        42.31   
           1         4      39954.04      False    A  151315        42.31   
           1         5      32229.38      False    A  151315        42.31   

                  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
Date       Store                                                           
2010-02-05 1           2.572        NaN        NaN        NaN        NaN   
           1           2.572        NaN        NaN        NaN        NaN   
           1           2.572        NaN        NaN        NaN        NaN   
           1           2.572        NaN        NaN        NaN        NaN   
           1           2.572        NaN        NaN        NaN        NaN   

                  MarkDown5         CPI  Unemployment  
Date       Store                                       
2010-02-05 1            NaN  211.096358         8.106  
           1            NaN  211.096358         8.106  
           1            NaN  211.096358         8.106  
           1            NaN  211.096358         8.106  
           1            NaN  211.096358         8.106

## Export as csv

In [14]:
merged_df.to_csv('Merged_Store_Data.csv')

## Summary
Nice work! You're working more and more independently through the workflow, and ensuring data integrity!